## Exemplo de instalação do Flask
O primeiro passo consiste em configurar e ativar o ambiente virtual, seguido da instalação do Flask.

In [ ]:
# Crie a pasta do projeto para organização
mkdir hello-flask && cd hello-flask
# Carregue o ambiente virtual para isolar as dependências do projeto
python3 -m venv .venv && source .venv/bin/activate
# Instale o Flask usando o gerenciador de pacotes pip
pip install Flask

## Executando o Flask
O arquivo principal de uma aplicação Flask, denominado `app.py`, é responsável por criar a instância da aplicação web e configurar sua execução, incluindo definições como porta, URL e modo de depuração. Nesse arquivo, você pode personalizar as configurações de sua aplicação e controlar como ela será executada e acessada.

No arquivo `app.py`, é possível configurar as rotas da aplicação Flask, que são definidas por meio de métodos Python decorados com `@app.route("/")`. Essas rotas determinam os diferentes caminhos ou URLs da aplicação, permitindo que o Flask saiba qual função executar quando um determinado caminho for acessado. Por exemplo:

In [ ]:
touch app.py

In [ ]:
# app.py
from flask import Flask
app = Flask(__name__)  # Cria a instância da aplicação


@app.route("/")  # Cria uma rota, para a raiz do projeto. (GET por padrão)
def hello_world():  # Método a ser executado ao navegar
    return 'Hello World!'


# Verifica se o script está sendo executado diretamente e executa a aplicação
if __name__ == '__main__':
    #  debug = True, reinicia automaticamente a cada mudança de arquivo
    #  mude a porta, caso ela estiver em uso
    app.run(debug=True, host='0.0.0.0', port=8000)

## Retorno em JSON
Você pode estar se perguntando: “Será que é complicado retornar um JSON API?”.

Na verdade, é muito simples com o Flask! A função `jsonify` é uma verdadeira ajuda, pois ela retorna um objeto `Response` do Flask com o cabeçalho `Content-Type: application/json`. Ou seja: boa parte dos detalhes de implementação é abstraído, tornando o desenvolvimento da sua API ainda mais rápido e eficiente.

Agora, vamos criar uma nova rota em localhost:8000/api, que irá retornar uma resposta em formato JSON usando a classe `flask.Response`.

In [ ]:
# app.py
from flask import Flask, jsonify

#...
@app.route("/api/")
def api_hello_world():
    return jsonify({'mensagem': 'Hello World!'})
#...

# Primeira API em Flask

Agora, vamos completar nossa API de cadastro de piadas realizando as 4 operações de um CRUD.

- Create (Post).
- Read (Get).
- Update (Put ou Patch).
- Delete (Delete).

Até este ponto, você viu como criar todas as funcionalidades dentro do arquivo `app.py`. No entanto, com base em experiências anteriores, você pode estar pensando que essa abordagem não é a mais adequada.

Chegou na hora de trazer de volta o conhecimento sobre _Controllers_, _Models_ e _Views_: a arquitetura **MVC**.

## Arquitetura MVC

Vamos dividir as responsabilidades de nosso _software_ em camadas:

- M (_Model_): lógica de negócios e interação com banco de dados (Objetos Principais).
- V (_View_): interface com a pessoa usuária (HTML).
- C (_Controller_): gerencia interação entre View e Model (Requisições HTTP).

O comando utilizado para criar os arquivos anteriormente foi extremamente útil, não acha?

Vamos utilizar mais um comando desses, na raiz do projeto, para finalizar a criação dos nossos arquivos:

Copiar

```bash
mkdir -p src/controllers && touch src/controllers/jokes_controller.py &&
mkdir -p src/models && touch src/models/db.py &&
mkdir -p src/models && touch src/models/abstract_model.py &&
mkdir -p src/models && touch src/models/joke_model.py
```

Com os arquivos em seus lugares, vamos completar a nossa `Joke API` e, assim, permitir as seguintes funcionalidades:

- Visualizar todas as piadas;
- Visualizar uma piada aleatória;
- Criar piadas;
- Editar piadas;
- Visualizar uma piada especifica;
- Excluir uma piada especifica.

## _Models_

As _models_ no padrão MVC são responsáveis por representar a lógica de negócios da aplicação e cuidar do acesso e da manipulação dos dados armazenados no banco de dados. Elas desempenham um papel fundamental como uma camada de abstração entre a aplicação e o banco de dados.

Usaremos o banco `MongoDB`, com a biblioteca `Pymongo` ao nosso lado.

Vamos realizar a conexão completando o arquivo `src/models/db.py`:

Copiar

```python
from pymongo import MongoClient
from os import environ

# Conecta no Mongo, pela variável ambiente definida no Docker Compose
client = MongoClient(environ.get("MONGO_URL"))

# Cria um banco de dados chamado db_chat
db = client.db_chat
```

Agora é hora de criar a model de piadas, chamada `JokeModel`, que será responsável por lidar com a comunicação com o banco de dados e atender às necessidades da nossa futura controller. Essa model será projetada levando em consideração as regras de negócio da nossa aplicação.

Como nosso foco é estudar o Flask sem aprofundar tanto no MongoDB no momento, vamos construir um código de fácil manutenção, abstraindo a lógica de comunicação com o banco de dados, em uma `AbstractModel`, que será herdada pela nossa _model_ `JokeModel`.

Assim, a `JokeModel` reaproveitará o código da classe-mãe `AbstractModel`, abstraindo a necessidade de conectar ao banco de dados.

Copiar

```python
# joke_model.py
import random
from .db import db
from .abstract_model import AbstractModel


# Herdado a classe Abstrata que domina o uso do Mongodb
class JokeModel(AbstractModel):
    # Define que a Coleção do Banco se chamara jokes.
    # Uma coleção é o equivalente a uma tabela no Mysql
    _collection = db["jokes"]

    # Nosso construtor receberá um dicionário (JSON)
    # para instanciar um objeto
    def __init__(self, json_data):
        super().__init__(json_data)

    # Retornar uma piada aleatória, é uma regra de negócio especifica
    # Fazendo sentido manter somente para a model Joke
    @classmethod
    def get_random(cls):
        data = cls.find()
        if not data:
            return
        return random.choice(data)

    # Define as regras de como o objeto JokeModel pode virar um Dict
    def to_dict(self):
        return {
            '_id': str(self.data['_id']),
            'joke': self.data['joke'],
        }
```

Agora, vamos criar a `src/models/abstract_model.py`.

No momento em que construímos a _model_ concreta `JokeModel`, preenchemos o atributo `_collection`. Ele representa uma coleção no banco MongoDB e é responsável por estabelecer a comunicação com o banco de dados, fornecendo as funcionalidades necessárias para nossa tarefa. Existem alguns métodos específicos do `Pymongo` que utilizaremos:

- `find`: Retorna uma lista com todos registros filtrados por uma _query_ (que é opcional);
- `find_one`: Equivalente ao `find`, mas retorna somente um registro;
- `find_one_and_update`: Procura um registro e o atualiza com os dados informados;
- `delete_one`: Excluir o primeiro registro que encontrar com a _query_ informada;
- `insert_one`: Insere um novo registro.

Uma _query_ no MongoDB segue o formato `JSON`. Para retornar o registro com o ID 1000, podemos utilizar a seguinte estrutura:

Copiar

```python
db["jokes"].find({"_id": 1000})
```

O equivalente à `Mysql` seria:

Copiar

```sql
"select * from jokes where id = 1000;"
```

Ao utilizar a classe `AbstractModel` como base para suas _models_, você está abstraindo o conhecimento específico do MongoDB e simplificando seu código. Isso significa que, ao herdar a `AbstractModel`, você automaticamente terá acesso aos métodos de busca, salvamento, atualização e exclusão de registros.

Copiar

```python
# abstract_model.py
from pymongo.collection import ReturnDocument, Collection


class AbstractModel:
    _collection: Collection = None

    def __init__(self, data: dict):
        self.data = data

    def save(self):
        result = self._collection.insert_one(self.data)
        inserted_document = self._collection.find_one(
            {"_id": result.inserted_id}
        )
        self.data = inserted_document
        return self.data

    def update(self, data: dict):
        # Por padrão o find_one_and_update retorna o estado anterior do
        # registro, por isso, utilizamos o ReturnDocument.AFTER para
        # retornar a versão pós atualização
        result = self._collection.find_one_and_update(
            {"_id": self.data["_id"]},
            {"$set": data},
            return_document=ReturnDocument.AFTER,
        )

        self.data = result
        return self.data

    def delete(self):
        self._collection.delete_one({"_id": self.data["_id"]})

    @classmethod
    def find(cls, query: dict = {}):
        data = cls._collection.find(query)
        return [cls(d) for d in data]

    @classmethod
    def find_one(cls, query: dict = {}):
        data = cls._collection.find_one(query)
        return cls(data) if data else None
```

# _Controllers_

Avançando na nossa construção, vamos criar outra camada essencial da arquitetura MVC: a _controller_. Ela é responsável por receber e processar as requisições HTTP, manipulando os dados necessários e retornando uma resposta apropriada para o cliente.

Nossa intenção é manter o arquivo `app.py` o mais limpo e organizado possível. Portanto, vamos separar a lógica de manipulação de requisições em _controllers_ individuais. Dessa forma, o `app.py` terá apenas a responsabilidade de iniciar o servidor e registrar os módulos das _controllers_.

Para registrar uma _controller_ no Flask, utilizaremos o método `app.register_blueprint`, que é disponibilizado pelo Flask e nos permite conectar os módulos da aplicação.

In [ ]:
from flask import Flask, jsonify
from flask import Flask
from controllers.jokes_controller import jokes_controller
import random
from os import environ
from waitress import serve

app = Flask(__name__)
app.register_blueprint(jokes_controller, url_prefix="/jokes")

# Removeremos nosso exemplo estático
joke_list = [
  "Por que o bombeiro não gosta de andar? <br> Porque ele socorre.",
  "Sabe como chama a sorveteria do Michel Teló? <br> Ice te Pego.",
  "Por que o espanador não luta caratê? <br> Porque ele luta capoeira"
]
@app.route("/api/joke")
def joke():
    return jsonify({'joke': random.choice(joke_list)})

def start_server(host: str = "0.0.0.0", port: int = 8000):
    if environ.get("FLASK_ENV") == "dev":
        # Servidor de desenvolvimento do Kit Werkzeug
        return app.run(debug=True, host=host, port=port)
    else:
        # Este é o waitress, otimizado para produção
        serve(app, host=host, port=port)

if __name__ == "__main__":
    start_server()


Agora, podemos finalmente criar nossa `Controller`, que retornará nossa API completa.

Usaremos alguns recursos interessantes do Flask:

- **Blueprint:** permite modularizar e reutilizar rotas em diferentes partes do código;
- **jsonify:** retorna uma resposta JSON para uma requisição, com o cabeçalho completo;
- **request:** encapsula a requisição HTTP, fornecendo os parâmetros da URL e o corpo da requisição.

In [ ]:
# jokes_controller.py
from bson import ObjectId
from flask import Blueprint, jsonify, request
from models.joke_model import JokeModel

jokes_controller = Blueprint("jokes", __name__)
# O primeiro parâmetro determina o nome da blueprint, já o segundo
# parâmetro é o caminho de importação do pacote com a blueprint,
# normalmente utilizamos __name__ referindo-se ao próprio módulo.
#
# Caso você tente registrar outra blueprint com o nome "jokes",
# você se irá se deparar com o seguinte erro:
#
# ValueError: The name 'jokes' is already registered for a different
# blueprint. Use 'name=' to provide a unique name.
#
# Ou seja: O nome 'jokes' já está registrado, mas podemos usar 'name='
# para fornecer um nome diferente para a sua nova Blueprint.


# Funções protegidas da camada de Controller
def _get_all_jokes():
    jokes = JokeModel.find()
    return [joke.to_dict() for joke in jokes]


def _get_joke(id: str):
    # ObjectId transforma uma string em ID do MongoDb
    return JokeModel.find_one({"_id": ObjectId(id)})


# ---------
# Rotas HTTP para nossa API
@jokes_controller.route("/", methods=["GET"])
def joke_index():
    jokes_list = _get_all_jokes()
    return jsonify(jokes_list)


@jokes_controller.route("/random", methods=["GET"])
def joke_random():
    joke = JokeModel.get_random()
    # Exemplo de Validação
    if joke is None:
        # O Flask entende que o número após o jsonify, representa o Status HTTP
        return jsonify({"error": "No jokes available"}), 404

    return jsonify(joke.to_dict()), 200


@jokes_controller.route("/", methods=["POST"])
def joke_post():
    new_joke = JokeModel(request.json)
    new_joke.save()
    return jsonify(new_joke.to_dict()), 201


@jokes_controller.route("/<id>", methods=["PUT"])
def joke_update(id: str):
    joke = _get_joke(id)
    # Exemplo de Validação
    if joke is None:
        return jsonify({"error": "Joke not found"}), 404
    joke.update(request.json)
    return jsonify(joke.to_dict()), 200


@jokes_controller.route("/<id>", methods=["GET"])
def joke_show(id: str):
    joke = _get_joke(id)
    if joke is None:
        return jsonify({"error": "Joke not found"}), 404
    return jsonify(joke.to_dict()), 200


@jokes_controller.route("/<id>", methods=["DELETE"])
def joke_delete(id: str):
    joke = _get_joke(id)
    if joke is None:
        return jsonify({"error": "Joke not found"}), 404

    joke.delete()
    return jsonify(joke.to_dict()), 204